In [2]:
import pickle
import os

import mne
from mne import create_info
from mne.io import RawArray

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

In [1]:
import numpy as np
print (np.__version__)

1.23.1


In [3]:
np.set_printoptions(precision=4)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
data_file = os.path.join(f'./data2/саша_открытые_глаза/OpenBCI-RAW-2023-11-01_17-00-13.txt')
with open(data_file, 'r') as in_f:
    header = True
    while header:
        line = next(in_f)
        if line[0] != '%':
            header = False
    data_arr = []
    for line in in_f:
        stripped_line = line.strip(' \n').split(', ')
        data_arr.append([float(stripped_line[0]),float(stripped_line[1]), float(stripped_line[2]), float(stripped_line[3]), float(stripped_line[4]), float(stripped_line[5]), float(stripped_line[6]),float(stripped_line[7]),float(stripped_line[8]), float(stripped_line[22]), stripped_line[24]])

    columns = ["Sample Index","EXG Channel 0", "EXG Channel 1", "EXG Channel 2", "EXG Channel 3", "EXG Channel 4", "EXG Channel 5", "EXG Channel 6", "EXG Channel 7", "Timestamp", "Timestamp Formatted"]
    dict_columns = {k: [v[i] for v in data_arr] for i, k in enumerate(columns)}

    df_file = pd.DataFrame.from_dict(dict_columns)
    df_file.to_csv(f'adelya.csv', index=False)



In [60]:
df = pd.read_csv("./data3/vadim.csv")

In [61]:
df.head(n=1000)

,Sample Index,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7,Timestamp,Timestamp Formatted
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.699550e+09,2023-11-09 20:10:31.202
1,1.0,39600.518596,47848.200542,32193.888091,36826.264778,79920.316329,37740.898161,-187500.022352,-187500.022352,1.699550e+09,2023-11-09 20:10:31.252
2,2.0,39605.860663,47849.049908,32199.096048,36827.248255,79924.540809,37748.229533,-187500.022352,-187500.022352,1.699550e+09,2023-11-09 20:10:31.252
3,3.0,39613.326146,47839.505713,32192.658745,36832.210342,79928.742937,37831.690947,-187500.022352,-187500.022352,1.699550e+09,2023-11-09 20:10:31.252
4,4.0,39613.907291,47830.497960,32185.483835,36832.456211,79924.227884,37858.803613,-187500.022352,-187500.022352,1.699550e+09,2023-11-09 20:10:31.252
...,...,...,...,...,...,...,...,...,...,...,...
995,207.0,38903.948832,47795.428073,32158.929963,37448.023253,79188.408457,37159.663398,-187500.022352,-187500.022352,1.699550e+09,2023-11-09 20:10:39.327
996,208.0,38910.944928,47801.016009,32178.554795,37459.400291,79196.522140,37150.454480,-187500.022352,-187500.022352,1.699550e+09,2023-11-09 20:10:39.343
997,209.0,38917.136361,47811.074294,32173.279783,37475.560603,79204.926396,37227.120963,-187500.022352,-187500.022352,1.699550e+09,2023-11-09 20:10:39.375
998,210.0,38936.940007,47813.644745,32165.479024,37484.881280,79217.376318,37294.176196,-187500.022352,-187500.022352,1.699550e+09,2023-11-09 20:10:39.375


In [62]:
df = df.drop(["Sample Index"], axis=1)
df = df.drop(["EXG Channel 6"], axis=1)
df = df.drop(["EXG Channel 7"], axis=1)
df = df.drop(["Timestamp"], axis=1)
df = df.drop(["Timestamp Formatted"], axis=1)

In [63]:
df.head()

,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,39600.518596,47848.200542,32193.888091,36826.264778,79920.316329,37740.898161
2,39605.860663,47849.049908,32199.096048,36827.248255,79924.540809,37748.229533
3,39613.326146,47839.505713,32192.658745,36832.210342,79928.742937,37831.690947
4,39613.907291,47830.497960,32185.483835,36832.456211,79924.227884,37858.803613


In [64]:

df.columns = ['O1' , 'Fp1', 'O2', 'Fp2', 'P8', 'F7']
df.head()

,O1,Fp1,O2,Fp2,P8,F7
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,39600.518596,47848.200542,32193.888091,36826.264778,79920.316329,37740.898161
2,39605.860663,47849.049908,32199.096048,36827.248255,79924.540809,37748.229533
3,39613.326146,47839.505713,32192.658745,36832.210342,79928.742937,37831.690947
4,39613.907291,47830.497960,32185.483835,36832.456211,79924.227884,37858.803613


In [65]:
def df_to_raw(df):
    sfreq = 255  #our OpenBCI headset sampling rate
    ch_names = list(df.columns)
    ch_types = ['eeg'] * (len(df.columns) - 1) + ['stim']
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

    df = df.T  #mne looks at the tranpose() format
    df[:-1] *= 1e-6  #convert from uVolts to Volts (mne assumes Volts data)

    info = create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)

    raw = mne.io.RawArray(df, info)
    raw.set_montage(ten_twenty_montage)

    #try plotting the raw data of its power spectral density
    raw.compute_psd().plot()

    return raw

In [66]:
raw = df_to_raw(df)

Creating RawArray with float64 data, n_channels=6, n_times=90981
    Range : 0 ... 90980 =      0.000 ...   356.784 secs
Ready.
Effective window size : 8.031 (s)


C:\Users\newYa\AppData\Local\Temp\ipykernel_2308\2824952393.py:13: RuntimeWarning: Not setting position of 1 stim channel found in montage:
['F7']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw.set_montage(ten_twenty_montage)
C:\Users\newYa\anaconda3\Lib\site-packages\mne\viz\utils.py:165: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  (fig or plt).show(**kwargs)


In [67]:
def get_freq(info):
    raw = info
    power, freq = raw.compute_psd().get_data(return_freqs=True)
    return freq


freq = get_freq(raw)

Effective window size : 8.031 (s)


In [68]:
band_names = np.array(['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta'])
filter_list = [[1,3],[4,7],[8,12],[13,30],[30,43], [4,13], [13,17]]
# band_names = np.array(['Delta', 'Theta', 'Alpha', 'Slow', 'Low_beta'])
# filter_list = [[1,3],[4,7],[8,12], [4,13], [13,17]]
bands = []
for filt in filter_list:
    pt = np.argwhere((freq >= filt[0]) & (freq <= filt[1])).reshape(-1)
    bands.append(pt)
bands = np.array(bands)
print(bands)

[array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
       dtype=int64)
 array([33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
        50, 51, 52, 53, 54, 55, 56], dtype=int64)
 array([65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
        82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96],
       dtype=int64)
 array([105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
        183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
        196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208,
        209, 210, 211, 212, 213,

C:\Users\newYa\AppData\Local\Temp\ipykernel_2308\2736347850.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bands = np.array(bands)


In [13]:
import mne

power,freq = raw.compute_psd().get_data(return_freqs=True)
data = power.mean(axis=1).reshape(1,-1)

Effective window size : 8.031 (s)


In [58]:
a_fp1, a_fp2 = None, None
a_t7, a_t8 = None, None
b_f3, b_f4 = None, None
b_t7, b_t8 = None, None

In [69]:
for index, band in enumerate(bands):
    if(band_names[index] == 'Alpha'):
        power,freq = raw.compute_psd().get_data(return_freqs=True)
        data = power[::,band].mean(axis=1).reshape(1,-1)

        a_fp1 = data[:,raw.ch_names.index('Fp1')]
        a_fp2= data[:,raw.ch_names.index('Fp2')]

        alpha_frontal = ((a_fp2 - a_fp1) / (a_fp2 + a_fp1)).reshape(1,-1)

        a_o1 = data[:,raw.ch_names.index('O1')]
        a_02 = data[:,raw.ch_names.index('O2')]

        alpha_temporal = ((a_02 - a_o1) / (a_02 + a_o1)).reshape(1,-1)

        alpha_asymmetry = alpha_frontal + alpha_temporal



        print(f'alpha_frontal {alpha_frontal}')
        print(f'alpha_temportal {alpha_temporal}')
        print(f'alpha_asymmetry {alpha_asymmetry}')

Effective window size : 8.031 (s)
alpha_frontal [[0.1746]]
alpha_temportal [[-0.1496]]
alpha_asymmetry [[0.025]]


In [178]:
a_fp1 = data[:,raw.ch_names.index('Fp1')]
a_fp2= data[:,raw.ch_names.index('Fp2')]

In [179]:
alpha_frontal = ((a_fp2 - a_fp1) / (a_fp2 + a_fp1)).reshape(1,-1)

In [180]:
print(f'alpha_frontal {alpha_frontal}')

alpha_frontal [[0.7444]]


In [181]:
a_o1 = data[:,raw.ch_names.index('O1')]
a_02 = data[:,raw.ch_names.index('O2')]

In [182]:
alpha_temporal = ((a_02 - a_o1) / (a_02 + a_o1)).reshape(1,-1)
print(f'alpha_temportal {alpha_temporal}')

alpha_temportal [[-0.9763]]


In [183]:
alpha_asymmetry = alpha_frontal + alpha_temporal
print(f'alpha_asymmetry {alpha_asymmetry}')

alpha_asymmetry [[-0.2319]]
